In [1]:
%load_ext autoreload
%autoreload 2
from ImagesProcess import ImagesProcess

imgs, labels = ImagesProcess.getImages()

error: OpenCV(4.1.0) /io/opencv/modules/imgproc/src/resize.cpp:3718: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [3]:
from sklearn.preprocessing import LabelEncoder
import os 
import pickle 
import numpy as np


def loadLabels():
    path = './labelsTrain'
    mylabels =list()
    for data in os.listdir(path):
        with open(os.path.join(path,data),'rb') as labels :
            mylabels = pickle.load(labels)
    return mylabels 


def scale_fit():
    path = './dataTrain/'
    lis = list()
    for data in os.listdir(path):
        with open(os.path.join(path,data),'rb') as style :
            mystyle = pickle.load(style)
            for image in mystyle:
                lis.append(image)
#     print(len(lis))
    lis = np.array(lis)
    
    return lis

In [4]:
import numpy as np
imgs = np.array(imgs)
labels = np.array(labels)

print(imgs.shape)
from keras.utils import to_categorical
y_binary = to_categorical(labels)

print(y_binary.shape)

(421, 100, 100, 3)


Using TensorFlow backend.


(421, 8)


In [5]:
from keras.layers import *
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense
#Image Dimensions
width=100
height=100
channels=3

model=Sequential()
model.add(Conv2D(32,(3,3),input_shape=(width,height,channels)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
#We are naming this layer, as we will extract features from it in future, JUST WAIT !!!
model.add(MaxPooling2D(pool_size=(2,2),name='feature_layer'))
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(8))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
#Let's prepare the data for 2D-CNN
X_Train=imgs.reshape(imgs.shape[0],100,100,3)
history=model.fit(X_Train,y_binary,epochs=33,batch_size=30,validation_split=0.33)

Instructions for updating:
Use tf.cast instead.
Train on 282 samples, validate on 139 samples
Epoch 1/33
282/282 [==============================] - 5s 16ms/step - loss: 13.8412 - acc: 0.1348 - val_loss: 14.6106 - val_acc: 0.0935
Epoch 2/33
282/282 [==============================] - 4s 15ms/step - loss: 13.7756 - acc: 0.1454 - val_loss: 14.6106 - val_acc: 0.0935
Epoch 3/33
282/282 [==============================] - 5s 16ms/step - loss: 13.0888 - acc: 0.1879 - val_loss: 14.6106 - val_acc: 0.0935
Epoch 4/33
282/282 [==============================] - 4s 15ms/step - loss: 13.4317 - acc: 0.1667 - val_loss: 14.6106 - val_acc: 0.0935
Epoch 5/33
282/282 [==============================] - 4s 16ms/step - loss: 14.0605 - acc: 0.1277 - val_loss: 14.6106 - val_acc: 0.0935
Epoch 6/33
282/282 [==============================] - 4s 16ms/step - loss: 13.6969 - acc: 0.1489 - val_loss: 14.6106 - val_acc: 0.0935
Epoch 7/33
282/282 [==============================] - 4s 16ms/step - loss: 14.0033 - acc: 0.1312

In [8]:
model.save("cnn.h5")

In [6]:
from keras.models import Model
new_model=Model(inputs=model.input,outputs=model.get_layer('feature_layer').output)

In [7]:
train_x=new_model.predict(X_Train)
#Convert back the labels
# train_y=[ np.where(r==1)[0][0] for r in y_binary ]
# #We are now going to have a single row for each example
# train_x=train_x.reshape(421,23*23*64)
print(train_x.shape)

# # (421, 23, 23, 64)

(421, 23, 23, 64)


In [33]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2 
print(train_x[2,:,:,0].shape)

for i in range(64):
    cv2.imwrite(str(i)+".jpg",train_x[i,:,:,0])

(23, 23)


In [38]:
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
knc=KNeighborsClassifier()
knc.fit(train_x,train_y)
knc.score(train_x,train_y)
# knn_predict=knc.predict(test_x)
# submissions=pd.DataFrame({"ImageId":list(range(1,len(knn_predict)+1))})
# submissions.to_csv("results.csv",index=False,header=True)

0.5866983372921615